In [6]:
!pip install evaluate

In [60]:
import pandas as pd
import numpy as np
import os
import random
import evaluate
from tqdm import tqdm

In [61]:
TRAIN_PATH = '/mnt/elice/dataset/train/'
TEST_PATH = '/mnt/elice/dataset/test/'

In [62]:
df = pd.read_csv(TRAIN_PATH + 'texts.csv')
raw_submission = pd.read_csv('raw_submission.csv')

In [66]:
def remove_duplicates(s):
    l = s.split(" ")
    while len(l) >= 2 and l[-1] == l[-2]:
        l = l[:-1]
    return " ".join(l)

In [67]:
print(remove_duplicates("같이 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진 사진"))
print(remove_duplicates("아 아 아 아 아"))

같이 사진
아


In [68]:
raw_submission["text"] = raw_submission["text"].progress_apply(remove_duplicates)

Pandas Apply:   0%|          | 0/6000 [00:00<?, ?it/s]

In [69]:
import re

def clean_text(text, remove_space=True):
    text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@\[\]^_\`{|}~\\\\]','', text)
    if remove_space:
        text = ''.join(text.split())
    return text

In [70]:
targets = list(map(lambda x : clean_text(x), set(df['text'].tolist()))) #train label값
threshold = 41 # text길이
cer_threshold = 0.5
metric = evaluate.load('cer')
def get_close(pred):
    if len(pred) >= threshold:
        return pred
    min_cer = 1e8
    index = -1
    for j in range(len(targets)):
        target = targets[j]
        if len(target) >= threshold:
            continue
        if len(target) > len(pred) * 2:
            continue
        if len(pred) > len(target) * 1.5:
            continue 
        cnt = 0
        for ch in target:
            if ch in pred:
                cnt += 1
        if cnt * 2 < len(target):
            continue
        cer = metric.compute(predictions=[pred], references=[target])
        if min_cer > cer:
            min_cer = cer
            index = j
    if min_cer < cer_threshold:
        pred = targets[index]
    return pred

In [71]:
import swifter
targets = list(map(lambda x : clean_text(x), set(df['text'].tolist()))) #train label값
preds = raw_submission['text'].apply(lambda x : clean_text(x))# predict 값

tqdm.pandas()
preds = preds.swifter.progress_bar(True).allow_dask_on_strings(enable=True).apply(get_close)
    
    

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/6000 [00:00<?, ?it/s]

In [72]:
submission = raw_submission
submission['text'] = preds

In [73]:
submission.to_csv('submission.csv', index=False)